# Part 2 - Looking at timeframes of project approvals
- OMG pls work. The idea is to create a machine learning model that will predict how long your approval will take.

In [343]:
import geopandas as gp
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [344]:
alvaras_file = 'C:/Users/Zaca/Documents/Datasets/lisbon/alvaras/alvaras.shp'

# approved construction permits
alvaras = gp.read_file(alvaras_file)

In [345]:
alvaras['OP_URBANIS'].value_counts()

Alteração                                                                  4048
Ampliação                                                                  2536
Alteração Durante a Execução da Obra                                       1505
Construção                                                                  610
Demolição                                                                   423
Demolição Autónoma                                                           61
Obras de Conservação                                                         58
Alteração ou Construção para instalação de antenas de radiocomunicações      38
Reconstrução                                                                 35
Renovação de Licença (artigo 72º do RJUE)                                    10
Operações Isentas de Licenciamento ou Autorização                             5
Name: OP_URBANIS, dtype: int64

In [346]:
# alvarás
# filter out entries that are only changes to a construction already in progress
#op = ['Alteração', 'Ampliação','Construção', 'Demolição', 'Alteração Durante a Execução da Obra ']
#alvaras = alvaras[alvaras['OP_URBANIS'].isin(op)]

In [347]:
alvaras['DATA_ENTRA'] = pd.to_datetime(alvaras['DATA_ENTRA'])
alvaras['DATA_ALVAR'] = pd.to_datetime(alvaras['DATA_ALVAR'])
alvaras['DEMORA'] = (alvaras['DATA_ALVAR'] - alvaras['DATA_ENTRA']).dt.days/7
alvaras['month'] = alvaras['DATA_ENTRA'].dt.month

In [348]:
cols = ['FREGUESIA', 'PROCEDIMEN', 'DEMORA', 'TIPO_ALVAR', 'month']
cat = ['FREGUESIA', 'PROCEDIMEN', 'TIPO_ALVAR']

In [349]:
alvaras

,OBJECTID,COD_SIG,IDTIPO,N_PROCESSO,DATA_ENTRA,TIPOLOGIA,MORADA,FREGUESIA,OP_URBANIS,ASSUNTO,PROCEDIMEN,TIPO_ALVAR,N_ALVARA,DATA_ALVAR,GlobalID,Shape__Are,Shape__Len,geometry,DEMORA,month
0,11,2304401003001,2,852/EDI/2007,2007-05-11,Edificação,R Chaby Pinheiro 20-20B,Avenidas Novas,Demolição,None,Licença,ED,48/ED/2010,2010-04-14,28ec034e-94ff-44db-bfe3-fef5daf43499,648.597656,107.073079,"POLYGON ((-9.14608 38.74486, -9.14592 38.74488...",152.714286,5
1,12,3202706022001,2,852/EDI/2008,2008-07-18,Edificação,R 2 (Bairro das Terras do Forno ou Novo de Bel...,Belém,Alteração Durante a Execução da Obra,Alterações Sujeitas a Licença,Licença,ADT,76/ADT/2009,2009-04-21,4e04a4d1-c0cb-4d5a-b996-42f22b135206,70.272949,33.608161,"POLYGON ((-9.20423 38.69874, -9.20416 38.69877...",39.571429,7
2,13,3100404009001,2,852/EDI/2009,2009-06-19,Edificação,R do Jardim do Regedor 10-32,Santa Maria Maior,Ampliação,None,Licença,EO,497/EO/2010,2010-11-03,63593f0a-0c94-4264-b9a3-36d9aaf1071e,662.744629,107.992681,"POLYGON ((-9.14066 38.71559, -9.14067 38.71560...",71.714286,6
3,14,4402803033001,2,852/EDI/2010,2010-08-11,Edificação,R Passos Manuel 19-,Arroios,Alteração Durante a Execução da Obra,Alteração ao projecto de arquitectura,Licença,ADT,66/ADT/2011,2011-05-23,e8bef844-7c30-47f3-8300-fde712218b97,554.964355,94.524498,"POLYGON ((-9.13697 38.72794, -9.13696 38.72779...",40.714286,8
4,15,0100901028001,2,852/EDI/2011,2011-08-16,Edificação,Estr da Cruz 30,Ajuda,Construção,None,Comunicação Prévia,CPREV,226/CPREV/2011,2011-12-21,6c479147-1940-402c-a8d2-4b660ec3d76b,296.551758,69.374560,"POLYGON ((-9.20697 38.71090, -9.20701 38.71086...",18.142857,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9324,21706,2601305004001,2,761/EDI/2013,2013-07-15,Edificação,R do Prior 3,Estrela,Ampliação,None,Licença,EO,516/EO/2015,2015-12-29,4aca370c-d116-446b-96fc-4ed9415b6667,232.627930,65.531750,"POLYGON ((-9.16175 38.70691, -9.16164 38.70693...",128.142857,7
9325,21722,2801101005001,2,772/EDI/2008,2008-07-04,Edificação,Pto do Tijolo 45-49,Misericórdia,Alteração,None,Licença,EO,142/EO/2010,2010-03-16,2bb09801-52dd-4f42-a4f5-adf0aa473ef9,64.000977,32.861046,"POLYGON ((-9.14734 38.71560, -9.14735 38.71565...",88.571429,7
9326,21723,0702507045001,2,772/EDI/2013,2013-07-17,Edificação,R Dom Francisco D`Eça 54,Beato,Alteração Durante a Execução da Obra,Alteração ao projecto de arquitectura,Comunicação Prévia,ADT,159/ADT/2013,2013-11-04,b4c81026-9890-4301-869f-484ee8f002df,80.615723,40.684366,"POLYGON ((-9.11197 38.73243, -9.11183 38.73244...",15.714286,7
9327,21724,2201303046001,2,772/EDI/2014,2014-07-14,Edificação,R da Cruz dos Poiais 38-38A,Misericórdia,Alteração Durante a Execução da Obra,Alteração ao projecto de arquitectura,Comunicação Prévia,CPREV,145/CPREV/2014,2014-08-14,cf4c62fb-ec88-4462-9add-b5001bf09b80,90.650391,43.103383,"POLYGON ((-9.15152 38.71164, -9.15150 38.71168...",4.428571,7


In [350]:
fun = alvaras[cols]

In [351]:
fun[cat].dtypes

FREGUESIA     object
PROCEDIMEN    object
TIPO_ALVAR    object
dtype: object

In [352]:
le = LabelEncoder()
fun[cat] = fun[cat].apply(le.fit_transform)

TypeError: argument must be a string or number

In [ ]:
sns.heatmap(fun.corr())

In [ ]:
X = fun.drop('DEMORA', axis=1)
y = fun['DEMORA']

In [ ]:
y

In [ ]:
# scale features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
X

In [ ]:
# make the split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# initialize linear regression model
lr = LinearRegression()

# fit training data
model = lr.fit(X_train, y_train)

In [ ]:
# get score
model.score(X_test, y_test)

In [ ]:
# initialize decision tree model
dtr = DecisionTreeRegressor(random_state=42)
model = dtr.fit(X_train, y_train)
model.score(X_test, y_test)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
# KNN regression
# initialize the model
for i in range(1, 50):
    knnr = KNeighborsRegressor(i)
    model = knnr.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    print(i, score)

In [ ]:
# random forest regression
for i in range(1, 15):
    rfr = RandomForestRegressor(max_depth=i, random_state=42)
    rfr.fit(X_train, y_train)
    score = rfr.score(X_test, y_test)
    print(i, score)

In [ ]:
from sklearn.metrics import mean_absolute_error
rfr = RandomForestRegressor(max_depth=8, random_state=42)
rfr.fit(X_train, y_train)
score = rfr.score(X_train, y_train)
print('Train score:', score)

y_pred_train = rfr.predict(X_train)
y_pred_test = rfr.predict(X_test)
mse_train = mean_absolute_error(y_train, y_pred_train)
mse_test = mean_absolute_error(y_test, y_pred_test)

print('mae train', mse_train)
print('mae test', mse_test)

In [ ]:
from sklearn.model_selection import cross_val_score
# cross validate the results of the random forest regressor
cv_rfr = cross_val_score(rfr, X, y, cv=10)
cv_rfr